<a href="https://colab.research.google.com/github/diegohugo570/backup-python/blob/main/02_O_Comando_Secreto_%7C_Clonagem_de_Tom_de_Voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai --quiet
!pip install httpx==0.27.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.34.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
from openai import OpenAI
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
def clonaTomDeVoz(artigos: list, openai_api_key: str):
  system_prompt = "You help users to mimic writing styles"
  if len(artigos) <= 6:
      trechosArtigos = "\n".join([f"<text>\n{artigo}\n</text>" for artigo in artigos[:1]])
      prompt1 = f"""
      Given the following sample text, please analyze and identify the author's distinctive stylistic elements in terms of tone, diction, sentence structure, burstiness, point of view, pace, literary devices and techniques, mood, perspective on time, personality, common enemies and text structure. Print them in a bullet list.
      {trechosArtigos}

      Return ONLY the stylistics.
      """

      client = OpenAI(api_key = OPENAI_API_KEY)
      messages=[{"role": "system", "content": system_prompt}]

      # Pegar o estilo (tom de comunicação, dicçao etc.)
      print("(1/6) Aprendendo o estilo de escrita...")
      messages.append({"role": "user", "content": prompt1})
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )
      stylistics = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": stylistics})

      # Pegar exemplos de trechos e comentar o que está sendo utilizado
      print("(2/6) Comentando exemplos de utilização do estilo...")
      prompt2 = f"""
      Now i need you to identify 5 segment of the text provided that captures the unique writing style of this author. Each segment can be up to 400 words verbatim and span multiple paragraph. Choose a good chunk of the text so that it can be understood in context

      After each segment add a short commentary on why it's chosen. wrap each segment with a example tag, ie:

      <example>
      segment
      </example>

      commentary

      <example>
      segment
      </example>

      commentary


      Return ONLY the content following this <example>segment</example> commentary format. ONLY this.
      The commentary must put emphasis on the elements of stylistcs used in each excerpt.
      """
      messages.append({"role": "user", "content": prompt2})
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )
      examples = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": examples})

      # Comparando a saída final com o exemplo de teste
      print("(3/6) Gerando o título para o último artigo")
      ultimoArtigo = artigos[-1]
      prompt_comparacao = f"Generate a 6 words title for the following article: {ultimoArtigo}. Return ONLY the title, nothing more."
      messages.append({"role": "user", "content": prompt_comparacao})
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )
      tituloUltimoArtigo = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": tituloUltimoArtigo})
      prompt3 = f"""
      I need you to mimic the writing style of an author. Write with the following stylistic elements and use the examples provided to understand to apply the style.

      ## Stylistic Elements

      {stylistics}

      ## Examples

      {examples}
      """
      prompt3_completo = prompt3 + f"""

      # IMPORTANT
      Now, using this style, write a article titled: "{tituloUltimoArtigo}". Return ONLY the article."""
      messages.append({"role": "user", "content": prompt3_completo})
      print("(4/6) Criando o artigo utilizando o primeiro estilo de comunicação")
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )
      articleExample = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": articleExample})

      # Comparando e refinando
      print("(5/6) Refinando o prompt")
      prompt4 = f"""
      Compare that with the following text and highlight key differences in stylistic elements:

      {articleExample}
      """
      messages.append({"role": "user", "content": prompt4})
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )
      completeToneCloned = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": completeToneCloned})


      # Final result
      print("(6/6) Final result")
      prompt5 = f"""
      Now, create a new stylistcs adding the key differences, to make it more complete! Merge in a more condense text, using only keywords.

      Original stylistics:
      {stylistics}

      Remember: write the new complete stylistics in bulletpoints.
      """
      messages.append({"role": "user", "content": prompt5})
      completion = client.chat.completions.create(
                  model="gpt-4o-mini",
                  messages = messages
                  )

      finalAnswer = """
      I need you to mimic the writing style of an author. Write with the following stylistic elements and use the examples provided to understand to apply the style.

      ## Stylistic Elements

"""
      finalAnswer += completion.choices[0].message.content
      messages.append({"role": "assistant", "content": finalAnswer})

      finalAnswer += "\n\n ## Examples\n" + examples

      finalAnswer += """\n\n## YOUR JOB\nWrite an article about "[TOPICO]". Here is the context: "[CONTEXTO]". Wirte in pt-BR."""

      with open("toneOfVoice.txt", "w") as arquivo:
          arquivo.write(finalAnswer)

  else:
      print("No máximo, 6 artigos de exemplo.")

In [ ]:
artigos = [
    """

    """,
    """

    """,
    """

    """,
    """

    """,
    """

    """
]

In [ ]:
clonaTomDeVoz(artigos = artigos, openai_api_key = OPENAI_API_KEY)

(1/6) Aprendendo o estilo de escrita...
(2/6) Comentando exemplos de utilização do estilo...
(3/6) Gerando o título para o último artigo
(4/6) Criando o artigo utilizando o primeiro estilo de comunicação
(5/6) Refinando o prompt
(6/6) Final result


## Repositório de Prompts (BÔNUS!)
Como bônus também, vou deixar aqui vários lugares onde vocês podem pesquisar exemplos de prompts para utilizar (e estudar!)

- https://github.com/0xeb/TheBigPromptLibrary/
- https://www.reddit.com/r/PromptEngineering/
- https://www.reddit.com/r/LocalLLaMA/
- https://github.com/jujumilk3/leaked-system-prompts
- https://github.com/CyberAlbSecOP/Awesome_GPT_Super_Prompting
- Artigo com os prompts: https://arxiv.org/pdf/2312.16171
- Prompting Caching: https://platform.openai.com/docs/guides/prompt-caching